# Batch Depth Prediction

This Colab will allow you to import an unlimited amount of pictures (jpg or not jpg, many formats supported) and transform all of them into depth maps predictions. You can then download all of them as a zip file.

How to Use:


1.   Right click "inputFolder" -> Import -> Select and load all your files 
2.   Run all the next 3 cells in order, to prepare the image processor
3.   Download "outputFolder", by running the 4th and last cell of this Colab

(credits: https://huggingface.co/spaces/nielsr/dpt-depth-estimation  )


# Run the next two cells in order, to load the image processor

In [ ]:
!pip install transformers
!pip install -r requirements.txt

In [14]:
from transformers import DPTFeatureExtractor, DPTForDepthEstimation
import torch
import numpy as np
import os
from PIL import Image
from google.colab import files

torch.hub.download_url_to_file('http://images.cocodataset.org/val2017/000000039769.jpg', 'cats.jpg')

feature_extractor = DPTFeatureExtractor.from_pretrained("Intel/dpt-large")
model = DPTForDepthEstimation.from_pretrained("Intel/dpt-large")

def process_image(image):
    # prepare image for the model
    encoding = feature_extractor(image, return_tensors="pt")
    
    # forward pass
    with torch.no_grad():
       outputs = model(**encoding)
       predicted_depth = outputs.predicted_depth
    
    # interpolate to original size
    prediction = torch.nn.functional.interpolate(
                        predicted_depth.unsqueeze(1),
                        size=image.size[::-1],
                        mode="bicubic",
                        align_corners=False,
                 ).squeeze()
    output = prediction.cpu().numpy()
    formatted = (output * 255 / np.max(output)).astype('uint8')
    img = Image.fromarray(formatted)
    return img
    
    return result

  0%|          | 0.00/169k [00:00<?, ?B/s]

Some weights of DPTForDepthEstimation were not initialized from the model checkpoint at Intel/dpt-large and are newly initialized: ['neck.fusion_stage.layers.0.residual_layer1.convolution2.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.weight', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.weight', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Run this next cell once you have imported all your pictures inside "inputFolder"

In [15]:
for filename in os.listdir("inputFolder"):
  if not filename.startswith("."): 
    myimg = Image.open("inputFolder/" + filename, mode='r')
    processedimg = process_image(myimg)
    processedimg.save("outputFolder/" + filename)

# Download the results by running this cell


In [17]:
!zip -r /content/outputFolder.zip /content/outputFolder
files.download('/content/outputFolder.zip')

updating: content/outputFolder/ (stored 0%)
updating: content/outputFolder/Sagrada Familia del roble.jpg (deflated 32%)
updating: content/outputFolder/.ipynb_checkpoints/ (stored 0%)
updating: content/outputFolder/rafael.jpg (deflated 41%)
updating: content/outputFolder/Caída en el camino del Calvario.jpg (deflated 41%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>